In [1]:
import com.griddynamics.dsl.ml.mldsl as mldsl
from com.griddynamics.dsl.ml.settings.profiles import PySparkJobProfile, Profile
from com.griddynamics.dsl.ml.settings.description import Platform

##### OLD JOB

In [2]:
%%py_script --name test_upgrade_job.py --path demo/scripts
# %py_load demo/scripts/test_upgrade_job.py
#!/usr/bin/python
import argparse
import json
import random 
import time
from datetime import datetime, timedelta

from pyspark import SparkContext
from pyspark.streaming import DStream, StreamingContext

# Define a callable source 
def generate_rdd(sc):
    reading = []
    for i in range(5):
        sensor_id = random.randint(1,100)
        reading.append(("sensor_" + str(sensor_id), random.random() * 1000, int((datetime.now().timestamp()))))
    return sc.parallelize(reading)

if __name__ == '__main__':
    sc = SparkContext(appName="test_new_functionality").getOrCreate()
    parser = argparse.ArgumentParser()
    parser.add_argument('--wait_delay', type=int, help='time between attempts')
    parser.add_argument('--wait_tries', type=int, help='number of attempts')
    parser.add_argument('--batch_interval', type=int, help='batch interval for streaming job')

    args = parser.parse_known_args()

    ssc = StreamingContext(sc, args[0].batch_interval)
    srs = [generate_rdd(sc) for i in range(1000)]
    lines = ssc.queueStream(srs)
    lines.pprint()
    ssc.start()
    ssc.awaitTermination()


Temporary path: /home/jovyan/work/data/.mldsl/test_upgrade_job.py


In [2]:
platform = Platform.GCP
logging={
    "driver_log_levels": {
        "com.example":4,"root":4, "com.google.cloud.pyspark":4
    }
}
profile = PySparkJobProfile(root_path='/home/jovyan/work/data/demo/scripts',
                                  bucket='ai4ops',
                                  project='gd-gcp-techlead-experiments',
                                  cluster='ai4ops',
                                  region='global',
                                  ai_region='us-central1',
                                  job_prefix='demo_job',
                                  job_async=True,
                           )
profile.logging = logging
Profile.set('TestJobProfile', profile)

In [4]:
%py_data -n test_upgrade_job.py -p TestJobProfile -pm $platform --batch_interval 1 -o gs://ai4ops/mldsl/data

Parameters string = <<<-n test_upgrade_job.py -p TestJobProfile -pm 1 --batch_interval 1 -o gs://ai4ops/mldsl/data>>>


Job with id demo_job_1585919338 was submitted to the cluster ai4ops


<IPython.core.display.JSON object>

In [ ]:
#Use job_demo_job_1585919338 instance to browse job properties.
job_demo_job_1585919338 = job_tracker['demo_job_1585919338']

##### JOB TO UPGRADE:

In [3]:
%py_script_open -n test_upgrade_job2.py -p demo/scripts -o dev

Temporary path: /home/jovyan/work/data/.mldsl/test_upgrade_job2.py


In [5]:
%%py_script -n my_validator.py -p demo/scripts
#!/usr/bin/python
from time import sleep
from com.griddynamics.dsl.ml.jobs.validator_job import Validator
from com.griddynamics.dsl.ml.helpers import *

class MyValidator(Validator):
    def validate(self, pattern="sensor_", wait_tries=5, wait_delay=5):
        try: 
            while wait_tries > 0:
                blobs = self.list_blobs()
                for blob in blobs:
                    read_output = blob.download_as_string().decode('utf-8')
                    if pattern not in read_output:
                        pass
                    else:
                        return True
                wait_tries -= 1
                sleep(wait_delay)
            return False
        except KeyboardInterrupt:
            return False
    
    def list_blobs(self):
        job_logs = self.job["driver_output_resource_uri"]
        prefix = job_logs.replace('gs://{}/'.format(self.session.bucket), '')
        storage_client = storage.Client(project=self.session.project_id)
        source_bucket = storage_client.get_bucket(self.session.bucket)
        blobs = source_bucket.list_blobs(prefix=prefix)
        return list(blobs)

Temporary path: /home/jovyan/work/data/.mldsl/my_validator.py


##### REPLACE OLD JOB WITH NEW ONE

In [6]:
%job_upgrade -n test_upgrade_job2.py -p TestJobProfile -pm $platform --batch_interval 1 -o gs://ai4ops/mldsl/data\
    --old_job_id demo_job_1585919607 -v MyValidator -vp  /home/jovyan/work/data/.mldsl/my_validator.py

Parameters string = <<<-n test_upgrade_job2.py -p TestJobProfile -pm 1 --batch_interval 1 -o gs://ai4ops/mldsl/data     --old_job_id demo_job_1585919607 -v MyValidator -vp  /home/jovyan/work/data/.mldsl/my_validator.py>>>


Job with id demo_job_1585920303 was submitted to the cluster ai4ops
Validation of new version job is successful
Canceling job: demo_job_1585919607
Job demo_job_1585919607 was successfully cancelled.


<IPython.core.display.JSON object>

In [ ]:
#Use job_demo_job_1585920303 instance to browse job properties.
job_demo_job_1585920303 = job_tracker['demo_job_1585920303']

##

##### NEW JOB DOESN'T MEET THE CONDITION(validation_pattern, wait_delay, wait_tries) IN APPROPRIATE TIME

In [4]:
%%py_script -n my_validator.py -p demo/scripts
#!/usr/bin/python
from time import sleep
from com.griddynamics.dsl.ml.jobs.validator_job import Validator
from com.griddynamics.dsl.ml.helpers import *

class MyValidator(Validator):
    def validate(self, pattern="sensor_", wait_tries=2, wait_delay=0.01):
        try: 
            while wait_tries > 0:
                blobs = self.list_blobs()
                for blob in blobs:
                    read_output = blob.download_as_string().decode('utf-8')
                    if pattern not in read_output:
                        pass
                    else:
                        return True
                wait_tries -= 1
                sleep(wait_delay)
            return False
        except KeyboardInterrupt:
            return False
    
    def list_blobs(self):
        job_logs = self.job['driver_output_resource_uri']
        prefix = job_logs.replace('gs://{}/'.format(self.session.bucket), '')
        storage_client = storage.Client(project=self.session.project_id)
        source_bucket = storage_client.get_bucket(self.session.bucket)
        blobs = source_bucket.list_blobs(prefix=prefix)
        return list(blobs)

Temporary path: /home/jovyan/work/data/.mldsl/my_validator.py


In [5]:
%job_upgrade -n test_upgrade_job2.py -p TestJobProfile -pm $platform --batch_interval 1 -o gs://ai4ops/mldsl/data\
    --old_job_id demo_job_1585920303 -v MyValidator -vp  /home/jovyan/work/data/demo/scripts/my_validator.py

Parameters string = <<<-n test_upgrade_job2.py -p TestJobProfile -pm 1 --batch_interval 1 -o gs://ai4ops/mldsl/data     --old_job_id demo_job_1585920303 -v MyValidator -vp  /home/jovyan/work/data/demo/scripts/my_validator.py>>>


Job with id demo_job_1585920813 was submitted to the cluster ai4ops
New job version doesn't meet the conditions or is invalid. Current working job version is demo_job_1585920303. Logs of failed job are in gs://ai4ops/google-cloud-dataproc-metainfo/2d057403-a13f-40fa-b785-43cf3d7e2bc3/jobs/demo_job_1585920813/driveroutput
Canceling job: demo_job_1585920813
Job demo_job_1585920813 was successfully cancelled.


<IPython.core.display.JSON object>

In [ ]:
#Use job_demo_job_1585920813 instance to browse job properties.
job_demo_job_1585920813 = job_tracker['demo_job_1585920813']